# Using Generative AI (Source Code)

In this notebook we will start looking at how we can take advantage of code generation for creating new node definitions.
While this is not definitive at least a starting point is possible as opposed to staring with nothing or performing 
code searches.

The initial interface will just be using text prompts using ChatGPT.

## Versioning is Important

Note that when generating it is quite important to explicitly specify the version of the library, tools, or shading language to avoid creating something unexpected. 

For now, at least with ChatGPT, just using it directly will **not** recognize current versions of MaterialX. It is possible to run this through an upgrade process however.

## Languages "Required"

To create a base level node the following languages should be implemented:

* `GLSL`
* `OSL`
* `NVIDIA MDL`
* `Apple MSL (Metal)`

`Vulkan` and `ESSL` are not required as they are created via generator variants of the`GLSL` generator.

If you have a custom language then there may be tools available for you to use for node graphs but at time
of writing unknown if there are tools for writing base nodes. E.g. There is a VEX translator for implementations
which are defined as functional nodegraphs.

## Creating a Graph From Source

This is still being experimented with but it is possible to a graph from source code via generative AI, though it may not match the syntax expected. A possible starting target is to create a Mermaid graph which can be written in text form.  



## 1. ChatGPT prompts 

In this example we will write a prompt for creating a "Gooch" shader. As the ultimate goal is to create a node we want this to be written as a callable function. We can start with GLSL as the intent is to make this work as a hardware shader.

The initial prompt is insufficient as it just writes code without even a callable function: `write glsl a gooch shader`

Certainly! Here's an example of a Gooch shader implemented in GLSL:

```glsl
#version 460

uniform vec3 lightDirection; // Direction of the light source
uniform vec3 surfaceColor;   // Color of the surface
uniform vec3 warmColor;      // Warm color for the shading
uniform vec3 coolColor;      // Cool color for the shading
uniform float alpha;         // Ambient color mixing factor
uniform float beta;          // Diffuse color mixing factor

in vec3 surfaceNormal;       // Surface normal
out vec4 fragColor;          // Output fragment color

void main() {
    vec3 N = normalize(surfaceNormal);
    vec3 L = normalize(lightDirection);
    float intensity = dot(N, L);

    vec3 warmTerm = mix(warmColor, coolColor, intensity);
    vec3 coolTerm = mix(coolColor, warmColor, intensity);

    vec3 diffuseColor = mix(warmTerm, coolTerm, beta);
    vec3 ambientColor = mix(coolColor, warmColor, alpha);

    vec3 finalColor = mix(diffuseColor, ambientColor, intensity);
    
    fragColor = vec4(finalColor * surfaceColor, 1.0);
}
```

In the above code, we have a Gooch shader that takes in several uniform variables: `lightDirection` represents the direction of the light source, `surfaceColor` is the color of the surface, `warmColor` is the warm color used for shading, `coolColor` is the cool color used for shading, `alpha` controls the mixing of ambient color, and `beta` controls the mixing of diffuse color.

The `surfaceNormal` is an input attribute representing the normal vector of the surface. The shader calculates the intensity of the light hitting the surface using the dot product of the surface normal and the light direction.

Based on the intensity, the shader interpolates between the warm and cool colors to create the diffuse color. It also interpolates between the cool and warm colors to generate the ambient color. The final color is obtained by mixing the diffuse and ambient colors based on the intensity.

The resulting color is then assigned to the `fragColor` output variable, which represents the final fragment color.

Please note that this code represents a basic implementation of the Gooch shading technique, and you may need to adjust it based on your specific requirements and rendering pipeline.


#### GLSL generation

The results are below. 
> Note that from the Web interface the text can be copied out directly as Markdown as is done here.

Certainly! Here's an example of a Gooch shader implemented in GLSL:

```glsl
uniform vec3 lightDirection; // Direction of the light source
uniform vec3 surfaceColor;   // Color of the surface
uniform vec3 warmColor;      // Warm color for the shading
uniform vec3 coolColor;      // Cool color for the shading
uniform float alpha;         // Ambient color mixing factor
uniform float beta;          // Diffuse color mixing factor

in vec3 surfaceNormal;       // Surface normal
out vec4 fragColor;          // Output fragment color

void main() {
    vec3 N = normalize(surfaceNormal);
    vec3 L = normalize(lightDirection);
    float intensity = dot(N, L);

    vec3 warmTerm = mix(warmColor, coolColor, intensity);
    vec3 coolTerm = mix(coolColor, warmColor, intensity);

    vec3 diffuseColor = mix(warmTerm, coolTerm, beta);
    vec3 ambientColor = mix(coolColor, warmColor, alpha);

    vec3 finalColor = mix(diffuseColor, ambientColor, intensity);
    
    fragColor = vec4(finalColor * surfaceColor, 1.0);
}
```

In the above code, we have a Gooch shader that takes in several uniform variables: `lightDirection` represents the direction of the light source, `surfaceColor` is the color of the surface, `warmColor` is the warm color used for shading, `coolColor` is the cool color used for shading, `alpha` controls the mixing of ambient color, and `beta` controls the mixing of diffuse color.

The `surfaceNormal` is an input attribute representing the normal vector of the surface. The shader calculates the intensity of the light hitting the surface using the dot product of the surface normal and the light direction.

Based on the intensity, the shader interpolates between the warm and cool colors to create the diffuse color. It also interpolates between the cool and warm colors to generate the ambient color. The final color is obtained by mixing the diffuse and ambient colors based on the intensity.

The resulting color is then assigned to the `fragColor` output variable, which represents the final fragment color.

Please note that this code represents a basic implementation of the Gooch shading technique, and you may need to adjust it based on your specific requirements and rendering pipeline.

After a few iterations we come up with something more suitable, making sure to use the coding "conventions" for code snippets and that we will need polymorphic versions for different output types -- in this case `color3` and `color4`.

The prompt is: `Write glsl version 4.6  function for a gooch shader with all parameters as arguments. Do not include an example using the function. Name the function "mx_gooch_" followed by the name of the output type. Prefix all parameter names with "mx_". Create both a 3 color and a 4 color version. `

Certainly! Here's the GLSL version 4.6 function for a Gooch shader with all parameters as arguments, including both a 3-color and a 4-color version. The function is named "mx_gooch_" followed by the name of the output type, and all parameter names are prefixed with "mx_":

```glsl
#version 460

// Gooch shader function for 3-color output
vec3 mx_gooch_vec3(
    vec3 mx_lightDirection,
    vec3 mx_surfaceNormal,
    vec3 mx_surfaceColor,
    vec3 mx_warmColor,
    vec3 mx_coolColor,
    float mx_alpha,
    float mx_beta
) {
    vec3 N = normalize(mx_surfaceNormal);
    vec3 L = normalize(mx_lightDirection);
    float intensity = dot(N, L);

    vec3 warmTerm = mix(mx_warmColor, mx_coolColor, intensity);
    vec3 coolTerm = mix(mx_coolColor, mx_warmColor, intensity);
    
    vec3 diffuseColor = mix(warmTerm, coolTerm, mx_beta);
    vec3 ambientColor = mix(mx_coolColor, mx_warmColor, mx_alpha);
    
    vec3 finalColor = mix(diffuseColor, ambientColor, intensity);
    
    return finalColor * mx_surfaceColor;
}

// Gooch shader function for 4-color output
vec4 mx_gooch_vec4(
    vec3 mx_lightDirection,
    vec3 mx_surfaceNormal,
    vec4 mx_surfaceColor,
    vec4 mx_warmColor,
    vec4 mx_coolColor,
    float mx_alpha,
    float mx_beta
) {
    vec3 N = normalize(mx_surfaceNormal);
    vec3 L = normalize(mx_lightDirection);
    float intensity = dot(N, L);

    vec4 warmTerm = mix(mx_warmColor, mx_coolColor, intensity);
    vec4 coolTerm = mix(mx_coolColor, mx_warmColor, intensity);
    
    vec4 diffuseColor = mix(warmTerm, coolTerm, mx_beta);
    vec4 ambientColor = mix(mx_coolColor, mx_warmColor, mx_alpha);
    
    vec4 finalColor = mix(diffuseColor, ambientColor, intensity);
    
    return finalColor * mx_surfaceColor;
}
```

In the above code, the `mx_gooch_vec3` function takes in the light direction (`mx_lightDirection`), surface normal (`mx_surfaceNormal`), surface color (`mx_surfaceColor`), warm color (`mx_warmColor`), cool color (`mx_coolColor`), alpha (`mx_alpha`), and beta (`mx_beta`) as parameters. It computes the Gooch shading equation for 3-color output and returns the final color.

Similarly, the `mx_gooch_vec4` function takes the same parameters but with `vec4` types for surface color, warm color, and cool color, and returns a 4-color output.

Please note that the code assumes the necessary uniforms and vertex attributes are provided externally and that you will need to integrate this function into a complete GLSL shader program for it to work properly.

Next we will derive the OSL, MDL, MSL versions from this with this prompt:

`Convert all of the previous GLSL shaders to versions written using Open Shading Language version 1.12, Metal Shading Language version 2.4, and NVIDIA MDL version 1.7 Do not include vertex programs`

Note that again we must take care to be specific about which language version to use as there is a tendency to use very old language versions (based on the amount of public code?)


Certainly! Here are the conversions of the previous GLSL shaders to Open Shading Language (OSL) version 1.12, Metal Shading Language (MSL) version 2.4, and NVIDIA Material Definition Language (MDL) version 1.7. Please note that vertex programs are not included in these conversions, as requested.

**Open Shading Language (OSL) version 1.12:**

```osl
shader mx_gooch(
    vector mx_lightDirection = vector(0.0, 1.0, 0.0),
    vector mx_surfaceNormal = vector(0.0, 0.0, 1.0),
    color mx_surfaceColor = color(1.0),
    color mx_warmColor = color(1.0, 0.75, 0.0),
    color mx_coolColor = color(0.0, 0.5, 1.0),
    float mx_alpha = 0.25,
    float mx_beta = 0.5
)
{
    normal N = normalize(mx_surfaceNormal);
    vector L = normalize(mx_lightDirection);
    float intensity = dot(N, L);

    color warmTerm = mix(mx_warmColor, mx_coolColor, intensity);
    color coolTerm = mix(mx_coolColor, mx_warmColor, intensity);

    color diffuseColor = mix(warmTerm, coolTerm, mx_beta);
    color ambientColor = mix(mx_coolColor, mx_warmColor, mx_alpha);

    color finalColor = mix(diffuseColor, ambientColor, intensity);

    Ci = finalColor * mx_surfaceColor;
}
```

**Metal Shading Language (MSL) version 2.4:**

```metal
#include <metal_stdlib>
using namespace metal;

fragment float4 mx_gooch(
    float3 mx_lightDirection [[buffer(0)]],
    float3 mx_surfaceNormal [[buffer(1)]],
    float4 mx_surfaceColor [[buffer(2)]],
    float4 mx_warmColor [[buffer(3)]],
    float4 mx_coolColor [[buffer(4)]],
    float mx_alpha [[buffer(5)]],
    float mx_beta [[buffer(6)]]
)
{
    float3 N = normalize(mx_surfaceNormal);
    float3 L = normalize(mx_lightDirection);
    float intensity = dot(N, L);

    float4 warmTerm = mix(mx_warmColor, mx_coolColor, intensity);
    float4 coolTerm = mix(mx_coolColor, mx_warmColor, intensity);

    float4 diffuseColor = mix(warmTerm, coolTerm, mx_beta);
    float4 ambientColor = mix(mx_coolColor, mx_warmColor, mx_alpha);

    float4 finalColor = mix(diffuseColor, ambientColor, intensity);

    return finalColor * mx_surfaceColor;
}
```

**NVIDIA Material Definition Language (MDL) version 1.7:**

```mdl
material mx_gooch(
    normal3d mx_surfaceNormal = normal3d(0.0, 0.0, 1.0),
    color3 mx_surfaceColor = color3(1.0),
    color3 mx_warmColor = color3(1.0, 0.75, 0.0),
    color3 mx_coolColor = color3(0.0, 0.5, 1.0),
    float mx_alpha = 0.25,
    float mx_beta = 0.5,
    normal3d mx_lightDirection = normal3d(0.0, 1.0, 0.0)
)
{
    normal3d

 N = normalize(mx_surfaceNormal);
    normal3d L = normalize(mx_lightDirection);
    float intensity = dot(N, L);

    color3 warmTerm = mix(mx_warmColor, mx_coolColor, intensity);
    color3 coolTerm = mix(mx_coolColor, mx_warmColor, intensity);

    color3 diffuseColor = mix(warmTerm, coolTerm, mx_beta);
    color3 ambientColor = mix(mx_coolColor, mx_warmColor, mx_alpha);

    color3 finalColor = mix(diffuseColor, ambientColor, intensity);

    result.color = finalColor * mx_surfaceColor;
}
```

Please note that in the MDL version, I made the `mx_lightDirection` parameter a normal3d to match the convention used in MDL.

### Creating a Node Graph

Next we will take this code which uses intrinsic functions and attempt to draw a diagram of how the node graph will look like.
We will start with a simple prompt, which gives a pretty poor result.

`Convert this function into a node graph diagram`



Unfortunately, as a text-based AI model, I am unable to generate graphical content directly. However, I can provide you with a textual representation of the node graph diagram for the Gooch shader function. 

```
            +--------------+
            | mx_lightDir  |
            +------+-------+
                   |
                   | Normalize
                   |
            +------v-------+
            | mx_surfaceN  |
            +------+-------+
                   |
                   | Normalize
                   |
            +------v-------+
            |   mx_N       |
            +------+-------+
                   |
                   | Dot product
                   |
            +------v-------+
            |   mx_L       |
            +------+-------+
                   |
                   | Dot product
                   |
            +------v-------+
            | mx_intensity |
            +------+-------+
                   |
                   | Mix
                   |
            +------+-------+
            | mx_warmColor |
            +------+-------+
                   |
                   | Mix
                   |
            +------+-------+
            | mx_coolColor |
            +------+-------+
                   |
                   | Mix
                   |
            +------+-------+
            | mx_beta      |
            +------+-------+
                   |
                   | Mix
                   |
            +------+-------+
            | mx_diffColor |
            +------+-------+
                   |
                   | Mix
                   |
            +------+-------+
            | mx_alpha     |
            +------+-------+
                   |
                   | Mix
                   |
            +------+-------+
            | mx_ambient   |
            +------+-------+
                   |
                   | Mix
                   |
            +------+-------+
            | mx_final     |
            +------+-------+
                   |
                   | Multiply
                   |
            +------+-------+
            | mx_surfaceC  |
            +--------------+
```

In the above representation, each node represents a specific operation or parameter involved in the Gooch shader function. The arrows indicate the flow of data between the nodes, representing the input and output connections. The rectangular boxes represent input parameters, and the rounded boxes represent individual operations or calculations. The final output is obtained by multiplying the `mx_final` color with the `mx_surfaceC` color.

This form will useful to look at does not lend itself to an API which creates and connects nodes. A next attempt will be the create a Mermaid graph. Again make sure to specify which version to use.

As something like Mermaid has not concept of ports the generation is mostly successful but requires hints to indicate what will translate into a node. e.g. intrinsics can translate into nodes.

The prompt used was:
`Convert the 3 color code into a Mermaid node graph diagram. Each math operation and intrinsic function should be a node`

Certainly! Here's the 3-color Gooch shader code converted into a Mermaid node graph diagram:

```mermaid
graph LR

input1(Light Direction) --> normalize1(Normalize)
input2(Surface Normal) --> normalize1
input3(Surface Color) --> multiply1(*)
input4(Warm Color) --> mix1(Mix)
input5(Cool Color) --> mix1
input6(Alpha) --> mix2(Mix)
input7(Beta) --> mix3(Mix)
normalize1 --> dot1(Dot Product)
dot1 --> intensity1(Intensity)
input4 --> mix2
input5 --> mix2
intensity1 --> mix2
input5 --> mix3
input4 --> mix3
intensity1 --> mix3
mix2 --> mix4(Mix)
mix3 --> mix4
mix4 --> mix5(Mix)
mix1 --> mix5
mix5 --> multiply1
multiply1 --> output1(Final Color)
normalize1 --> multiply1
input6 --> mix3
input7 --> mix4
```

In the diagram, each input parameter is represented by a node, and each math operation or intrinsic function is also represented as a node. The arrows indicate the flow of data from one node to another. The final output is represented by the "Final Color" node.

Please note that this diagram represents the flow of data and operations in the Gooch shader code, but it does not include any information about the specific values or constants used in the shader.

<image src="./data/ai_generated_gooch_graph.svg">

This is slightly better even after many iterations it was not possible to get all the nodes and inputs present and connected properly.

A "divide-and-conquer" approach was tried next by compartmentalizing each intrsinsic / basic function to a function with the prompt:
`Write a set of functions to create a shader graph for the 3 color GLSL code. `


To create a shader graph for the 3-color Gooch shader in GLSL, we can break down the calculations into separate functions. Here's a set of functions that represent the different parts of the shader code:

```glsl
#version 460

vec3 mx_computeIntensity(vec3 mx_surfaceNormal, vec3 mx_lightDirection) {
    vec3 N = normalize(mx_surfaceNormal);
    vec3 L = normalize(mx_lightDirection);
    return vec3(dot(N, L));
}

vec3 mx_computeWarmTerm(vec3 mx_intensity, vec3 mx_warmColor, vec3 mx_coolColor) {
    return mix(mx_warmColor, mx_coolColor, mx_intensity);
}

vec3 mx_computeCoolTerm(vec3 mx_intensity, vec3 mx_warmColor, vec3 mx_coolColor) {
    return mix(mx_coolColor, mx_warmColor, mx_intensity);
}

vec3 mx_computeDiffuseColor(vec3 mx_warmTerm, vec3 mx_coolTerm, float mx_beta) {
    return mix(mx_warmTerm, mx_coolTerm, mx_beta);
}

vec3 mx_computeAmbientColor(vec3 mx_coolColor, vec3 mx_warmColor, float mx_alpha) {
    return mix(mx_coolColor, mx_warmColor, mx_alpha);
}

vec3 mx_computeFinalColor(vec3 mx_diffuseColor, vec3 mx_ambientColor, float mx_intensity) {
    return mix(mx_diffuseColor, mx_ambientColor, mx_intensity);
}

vec3 mx_gooch(vec3 mx_lightDirection, vec3 mx_surfaceNormal, vec3 mx_surfaceColor, vec3 mx_warmColor, vec3 mx_coolColor, float mx_alpha, float mx_beta) {
    vec3 intensity = mx_computeIntensity(mx_surfaceNormal, mx_lightDirection);
    vec3 warmTerm = mx_computeWarmTerm(intensity, mx_warmColor, mx_coolColor);
    vec3 coolTerm = mx_computeCoolTerm(intensity, mx_warmColor, mx_coolColor);
    vec3 diffuseColor = mx_computeDiffuseColor(warmTerm, coolTerm, mx_beta);
    vec3 ambientColor = mx_computeAmbientColor(mx_coolColor, mx_warmColor, mx_alpha);
    vec3 finalColor = mx_computeFinalColor(diffuseColor, ambientColor, intensity);
    return finalColor * mx_surfaceColor;
}
```

In this set of functions, we have the following:

- `mx_computeIntensity` calculates the intensity of the light based on the surface normal and light direction.
- `mx_computeWarmTerm` computes the warm term color using the provided warm and cool colors, along with the intensity.
- `mx_computeCoolTerm` calculates the cool term color using the warm and cool colors and the intensity.
- `mx_computeDiffuseColor` blends the warm and cool terms based on the beta value.
- `mx_computeAmbientColor` interpolates between the cool and warm colors using the alpha value.
- `mx_computeFinalColor` combines the diffuse and ambient colors based on the intensity.
- `mx_gooch` is the main function that takes all the parameters and computes the final color using the previously defined functions.

By organizing the shader calculations into separate functions, you can easily create a shader graph by connecting these functions together based on the inputs and outputs.



`Create a set of instructions for creating nodes and connections from the previous 3 color GLSL code. Create nodes for all input parameters.`

Certainly! Here's a set of instructions for creating nodes and connections based on the previous 3-color Gooch shader GLSL code, including nodes for all input parameters:

1. Create the following nodes:
   - `Surface Normal` (Vector3)
   - `Light Direction` (Vector3)
   - `Surface Color` (Vector3)
   - `Warm Color` (Vector3)
   - `Cool Color` (Vector3)
   - `Alpha` (Float)
   - `Beta` (Float)
   - `Intensity` (Vector3)
   - `Warm Term` (Vector3)
   - `Cool Term` (Vector3)
   - `Diffuse Color` (Vector3)
   - `Ambient Color` (Vector3)
   - `Final Color` (Vector3)

2. Set the values of the input parameter nodes according to your requirements:
   - Set the `Surface Normal` node to the surface normal value.
   - Set the `Light Direction` node to the light direction value.
   - Set the `Surface Color` node to the surface color value.
   - Set the `Warm Color` node to the warm color value.
   - Set the `Cool Color` node to the cool color value.
   - Set the `Alpha` node to the alpha value.
   - Set the `Beta` node to the beta value.

3. Create connections between the nodes:
   - Connect the `Surface Normal` node to the `Intensity` node.
   - Connect the `Light Direction` node to the `Intensity` node.
   - Connect the `Intensity` node to both the `Warm Term` and `Cool Term` nodes.
   - Connect the `Intensity` node to the `Diffuse Color` node.
   - Connect the `Warm Color` and `Cool Color` nodes to the `Warm Term` and `Cool Term` nodes, respectively.
   - Connect the `Beta` node to the `Diffuse Color` node.
   - Connect the `Cool Color` and `Warm Color` nodes to the `Ambient Color` node.
   - Connect the `Alpha` node to the `Ambient Color` node.
   - Connect the `Diffuse Color` and `Ambient Color` nodes to the `Final Color` node.
   - Connect the `Surface Color` node to the `Final Color` node.

4. You have now created a shader graph that represents the computation flow of the 3-color Gooch shader. You can now use this graph to generate the shader code or further manipulate the graph to add additional functionality or effects.

Note: Depending on the software or tool you are using for creating shader graphs, the specific steps for creating nodes and connections may vary. However, the overall concept of creating nodes for each input parameter and connecting them according to the shader code logic remains the same.

Then these use this to try and create a MaterialX graph. The is roughly correct but for some reason it does not use the correct syntax for creating nodes and instead creates generic `nodes`. Also the connection syntax is completely incorrect.

`Use these instructions to create a Version 1.38 MaterialX node graph. Create  separate MaterialX nodegraph representations for each additional functions.`

Certainly! Here's how you can create MaterialX node graphs for each function of the 3-color Gooch shader:

**1. MaterialX Node Graph for `mx_computeIntensity` function:**
```
<materialx version="1.38">
  <graph name="mx_computeIntensity">
    <input name="mx_surfaceNormal" type="vector3" />
    <input name="mx_lightDirection" type="vector3" />
    <output name="mx_intensity" type="vector3" />

    <node name="normalize" nodetype="normalize">
      <input name="in" type="vector3" value="mx_surfaceNormal" />
    </node>

    <node name="normalizeLight" nodetype="normalize">
      <input name="in" type="vector3" value="mx_lightDirection" />
    </node>

    <node name="dotProduct" nodetype="dotproduct">
      <input name="in1" type="vector3" value="normalize.out" />
      <input name="in2" type="vector3" value="normalizeLight.out" />
    </node>

    <outputconnect source="dotProduct.out" destination="mx_intensity" />
  </graph>
</materialx>
```

**2. MaterialX Node Graph for `mx_computeWarmTerm` function:**
```xml
<materialx version="1.38">
  <graph name="mx_computeWarmTerm">
    <input name="mx_intensity" type="vector3" />
    <input name="mx_warmColor" type="vector3" />
    <input name="mx_coolColor" type="vector3" />
    <output name="mx_warmTerm" type="vector3" />

    <node name="mix" nodetype="mix">
      <input name="in1" type="vector3" value="mx_warmColor" />
      <input name="in2" type="vector3" value="mx_coolColor" />
      <input name="in3" type="vector3" value="mx_intensity" />
    </node>

    <outputconnect source="mix.out" destination="mx_warmTerm" />
  </graph>
</materialx>
```

**3. MaterialX Node Graph for `mx_computeCoolTerm` function:**
```xml
<materialx version="1.38">
  <graph name="mx_computeCoolTerm">
    <input name="mx_intensity" type="vector3" />
    <input name="mx_warmColor" type="vector3" />
    <input name="mx_coolColor" type="vector3" />
    <output name="mx_coolTerm" type="vector3" />

    <node name="mix" nodetype="mix">
      <input name="in1" type="vector3" value="mx_coolColor" />
      <input name="in2" type="vector3" value="mx_warmColor" />
      <input name="in3" type="vector3" value="mx_intensity" />
    </node>

    <outputconnect source="mix.out" destination="mx_coolTerm" />
  </graph>
</materialx>
```

**4. MaterialX Node Graph for `mx_computeDiffuseColor` function:**
```xml
<materialx version="1.38">
  <graph name="mx_computeDiffuseColor">
    <input name="mx_warmTerm" type="vector3" />
    <input name="mx_coolTerm" type="vector3" />
    <input name="mx_beta" type="float" />
    <output name="mx_diffuseColor" type="vector3"

 />

    <node name="mix" nodetype="mix">
      <input name="in1" type="vector3" value="mx_warmTerm" />
      <input name="in2" type="vector3" value="mx_coolTerm" />
      <input name="in3" type="float" value="mx_beta" />
    </node>

    <outputconnect source="mix.out" destination="mx_diffuseColor" />
  </graph>
</materialx>
```

**5. MaterialX Node Graph for `mx_computeAmbientColor` function:**
```xml
<materialx version="1.38">
  <graph name="mx_computeAmbientColor">
    <input name="mx_coolColor" type="vector3" />
    <input name="mx_warmColor" type="vector3" />
    <input name="mx_alpha" type="float" />
    <output name="mx_ambientColor" type="vector3" />

    <node name="mix" nodetype="mix">
      <input name="in1" type="vector3" value="mx_coolColor" />
      <input name="in2" type="vector3" value="mx_warmColor" />
      <input name="in3" type="float" value="mx_alpha" />
    </node>

    <outputconnect source="mix.out" destination="mx_ambientColor" />
  </graph>
</materialx>
```

**6. MaterialX Node Graph for `mx_computeFinalColor` function:**
```xml
<materialx version="1.38">
  <graph name="mx_computeFinalColor">
    <input name="mx_diffuseColor" type="vector3" />
    <input name="mx_ambientColor" type="vector3" />
    <input name="mx_intensity" type="vector3" />
    <output name="mx_finalColor" type="vector3" />

    <node name="mix" nodetype="mix">
      <input name="in1" type="vector3" value="mx_diffuseColor" />
      <input name="in2" type="vector3" value="mx_ambientColor" />
      <input name="in3" type="vector3" value="mx_intensity" />
    </node>

    <outputconnect source="mix.out" destination="mx_finalColor" />
  </graph>
</materialx>
```

Each of the above MaterialX node graphs represents a specific function from the 3-color Gooch shader. You can now use these node graphs to create a complete MaterialX graph or integrate them into your existing MaterialX project as needed.

Things are slightly better when trying to create version 1.37 syntax. There is no feedback that it does not understand version 1.38, so it seems to revert back to some really old version.

## 2. Github Copilot 

Copilot  supports both <a href="https://code.visualstudio.com/docs/editor/artificial-intelligence#_chat-view" target="_blank">prompt</a> and in-code for Visual Studio code. Both have been installed here.
* For in-code generation, a comment can be added as "commands" for generating code. 
* It is not very "smart" for using local libraries. TBD if there is a way to scan in local code to aid in code
generation.
* It is worthwhile for the case of MaterialX to allow usage of "public" code when generating. 
* It is still not very good a recognizing the arguments required for a interface as it seems to use the bare minimum of arguments.
* Oft times the incorrect code is currently being generated which makes it of very limited use for now as only very logic
can be created. This may be due to limited usage of Python and thus availability of Python examples ?

### Code-Pilot Code Generation

In [1]:
import MaterialX as mx

# Use the version of MaterialX that is installed with the Python package manager
print(mx.__version__)

# Create a document.
doc = mx.createDocument()

# Find a child node named "abc" in the document. 
node = doc.getChild("abc")

# If the node is found the delete the node
if node:
    doc.removeChild(node)
    
# Create a node graph called "myNodeGraph"
nodeGraph = doc.addNodeGraph("myNodeGraph")

# Add inputs and nodes to the graph which add 2 color3 values together and output the result using MaterialX 1.37 syntax
### This simple task fails
#input1 = nodeGraph.addInput("input1", "color3")
#input2 = nodeGraph.addInput("input2", "color3")
#addNode = nodeGraph.addNode("add", "add")
#output = nodeGraph.addOutput("output", "color3")
#nodeGraph.connect(input1, addNode.getInput("in1"))
#nodeGraph.connect(input2, addNode.getInput("in2"))
#nodeGraph.connect(addNode.getOutput("out"), output)

# Check if myNodeGraph node graph exists in the document. If it does not exist then create it
graph = doc.getNodeGraph("myNodeGraph")
if not nodeGraph:
    graph = doc.addNodeGraph("myNodeGraph")

# Add 2 vector3 interface inputs to myGraph
graph.addInput("input1", "vector3")
graph.addInput("input2", "vector3")
# Add an add node to myGraph
graph.addNode("add", "add")
# Add of the input children of add
graph.getNode("add").addInput("in1", "vector3")
graph.getNode("add").addInput("in2", "vector3")
# Add an output to myGraph
graph.addOutput("output", "vector3")
# Connect the first input on add to input1 by declaring an interfacename attribute on the first input of add
graph.getNode("add").getInput("in1").setAttribute("interfaceName", "input1")
# Connect the second input on add to input1 by declaring an interfacename attribute on the second input of add
graph.getNode("add").getInput("in2").setAttribute("interfaceName", "input2")
# Connect the output of add node to output on the node graph using the "nodename" attribute to specify the node to connect to.
graph.getOutput("output").setAttribute("nodename", "add")

# validate the document
doc.validate()

# use pretty print utility to print the node graph. this fails...
#mx.writeToXmlFile(doc, "myGraph.mtlx", "myGraph")
# print the node graph mygraph

# add nodes to add 2 vector3 values together
#input1 = graph.addInput("input1", "vector3")
#input2 = graph.addInput("input2", "vector3")
#addNode = graph.addNode("add", "add")
#output = graph.addOutput("output", "vector3")

# Connect the inputs of addNode as interface inputs on myGraph using the version of MaterialX from the Python package manager
## Still wrong.
#graph.connect(input1, addNode.getInput("in1"))
#graph.connect(input2, addNode.getInput("in2"))
#graph.connect(addNode.getOutput("out"), output)

# Upgrade the document to version 1.38. This produces bogus code.
#mx.upgradeDocument(doc, "1.38")

# Iterate through all nodes in nodeGraph and print out their path
for node in nodeGraph.getNodes():
    print(node.getNamePath())


# Get the node definition for noise
noiseDef = doc.getNodeDef("noise")
# if the definition exists add a noise node to the node graph. This is wrong.
if noiseDef:
    nodeGraph.addNodeInstance("noise", "noise")

# Write document to string
xmlString = mx.writeToXmlString(doc)
print(xmlString)



1.39.2
myNodeGraph/add
<?xml version="1.0"?>
<materialx version="1.39">
  <nodegraph name="myNodeGraph">
    <input name="input1" type="vector3" />
    <input name="input2" type="vector3" />
    <add name="add" type="color3">
      <input name="in1" type="vector3" interfaceName="input1" />
      <input name="in2" type="vector3" interfaceName="input2" />
    </add>
    <output name="output" type="vector3" nodename="add" />
  </nodegraph>
</materialx>



## Copilot  Chat

## 3. JSON Definition Example

Try to recreate a JSON schema based on MaterialX using ChatGPT.

Certainly! Here's an updated version of the JSON schema that allows a node to have a category, group membership, and a string reference to an implementation:

```json
{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "definitions": {
    "BaseNode": {
      "type": "object",
      "properties": {
        "id": {
          "type": "string"
        },
        "category": {
          "type": "string"
        },
        "group": {
          "type": "string"
        },
        "implementation": {
          "type": "string"
        },
        "inputs": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/InputPort"
          }
        },
        "outputs": {
          "type": "array",
          "items": {
            "$ref": "#/definitions/OutputPort"
          }
        }
      },
      "required": ["id", "category", "group", "implementation", "inputs", "outputs"]
    },
    "NodeGraph": {
      "allOf": [
        {
          "$ref": "#/definitions/BaseNode"
        },
        {
          "type": "object",
          "properties": {
            "name": {
              "type": "string"
            },
            "nodes": {
              "type": "array",
              "items": {
                "oneOf": [
                  { "$ref": "#/definitions/Node" },
                  { "$ref": "#/definitions/NodeGraph" }
                ]
              }
            },
            "edges": {
              "type": "array",
              "items": {
                "$ref": "#/definitions/Edge"
              }
            }
          },
          "required": ["name", "nodes", "edges"]
        }
      ]
    },
    "Node": {
      "allOf": [
        {
          "$ref": "#/definitions/BaseNode"
        },
        {
          "type": "object",
          "properties": {
            "name": {
              "type": "string"
            }
          },
          "required": ["name"]
        }
      ]
    },
    "Port": {
      "type": "object",
      "properties": {
        "id": {
          "type": "string"
        },
        "name": {
          "type": "string"
        },
        "type": {
          "enum": ["color3", "vector3", "vector2", "vector4", "color4", "matrix3x3", "matrix4x4"]
        }
      },
      "required": ["id", "name", "type"]
    },
    "InputPort": {
      "allOf": [
        {
          "$ref": "#/definitions/Port"
        },
        {
          "properties": {
            "type": {
              "not": {
                "enum": ["color4"]
              }
            }
          }
        }
      ]
    },
    "OutputPort": {
      "allOf": [
        {
          "$ref": "#/definitions/Port"
        },
        {
          "properties": {
            "type": {
              "not": {
                "enum": ["color4"]
              }
            }
          }
        }
      ]
    },
    "Edge": {
      "type": "object",
      "properties": {
        "from": {
          "oneOf": [
            {
              "type": "string"
            },
            {
              "$ref": "#/definitions/PortConnection"
            }
          ]
        },
        "to": {
          "oneOf": [
            {
              "type": "string"
            },
            {
              "$ref": "#/

definitions/PortConnection"
            }
          ]
        }
      },
      "required": ["from", "to"]
    },
    "PortConnection": {
      "type": "object",
      "properties": {
        "nodeId": {
          "type": "string"
        },
        "portId": {
          "type": "string"
        }
      },
      "required": ["nodeId", "portId"]
    }
  }
}
```

In this updated schema, the `BaseNode` definition has been expanded to include three new properties: `category`, `group`, and `implementation`. The `category` property represents the category of the node, the `group` property represents the group membership of the node, and the `implementation` property stores a string reference to the implementation of the node.

Both the `Node` and `NodeGraph` schemas inherit from the `BaseNode` schema and include these new properties.

You can now use this updated schema to represent nodes with categories, group memberships, and references to their implementations.

**and an example graph**

> Create an example node network using this schema which computes the distance between 2 3d points. Assume that each math operator is a node. Make the category the operator name. Set input port names to use the prefix "in" and output port names the prefix "output"

Certainly! Here's an example of a node network that computes the distance between two 3D points using the provided schema:

```json
{
  "nodes": [
    {
      "id": "node1",
      "category": "Add",
      "group": "Math",
      "implementation": "addition.node",
      "inputs": [
        {
          "id": "in1",
          "name": "inVectorA",
          "type": "vector3"
        },
        {
          "id": "in2",
          "name": "inVectorB",
          "type": "vector3"
        }
      ],
      "outputs": [
        {
          "id": "output1",
          "name": "outputVector",
          "type": "vector3"
        }
      ]
    },
    {
      "id": "node2",
      "category": "Subtract",
      "group": "Math",
      "implementation": "subtraction.node",
      "inputs": [
        {
          "id": "in1",
          "name": "inVectorA",
          "type": "vector3"
        },
        {
          "id": "in2",
          "name": "inVectorB",
          "type": "vector3"
        }
      ],
      "outputs": [
        {
          "id": "output1",
          "name": "outputVector",
          "type": "vector3"
        }
      ]
    },
    {
      "id": "node3",
      "category": "Power",
      "group": "Math",
      "implementation": "power.node",
      "inputs": [
        {
          "id": "in1",
          "name": "inValue",
          "type": "vector3"
        },
        {
          "id": "in2",
          "name": "inPower",
          "type": "vector3"
        }
      ],
      "outputs": [
        {
          "id": "output1",
          "name": "outputValue",
          "type": "vector3"
        }
      ]
    },
    {
      "id": "node4",
      "category": "Square Root",
      "group": "Math",
      "implementation": "square_root.node",
      "inputs": [
        {
          "id": "in1",
          "name": "inValue",
          "type": "vector3"
        }
      ],
      "outputs": [
        {
          "id": "output1",
          "name": "outputValue",
          "type": "vector3"
        }
      ]
    },
    {
      "id": "node5",
      "category": "Distance",
      "group": "Math",
      "implementation": "distance.node",
      "inputs": [
        {
          "id": "in1",
          "name": "inPointA",
          "type": "vector3"
        },
        {
          "id": "in2",
          "name": "inPointB",
          "type": "vector3"
        }
      ],
      "outputs": [
        {
          "id": "output1",
          "name": "outputDistance",
          "type": "vector3"
        }
      ]
    }
  ],
  "edges": [
    {
      "from": {
        "nodeId": "node1",
        "portId": "output1"
      },
      "to": {
        "nodeId": "node2",
        "portId": "in1"
      }
    },
    {
      "from": {
        "nodeId": "node3",
        "portId": "output1"
      },
      "

to": {
        "nodeId": "node2",
        "portId": "in2"
      }
    },
    {
      "from": {
        "nodeId": "node2",
        "portId": "output1"
      },
      "to": {
        "nodeId": "node4",
        "portId": "in1"
      }
    },
    {
      "from": {
        "nodeId": "node4",
        "portId": "output1"
      },
      "to": {
        "nodeId": "node5",
        "portId": "in1"
      }
    }
  ]
}
```

In this example, the node network consists of five nodes: "Add," "Subtract," "Power," "Square Root," and "Distance." Each node is placed in the "Math" group and has an implementation associated with it.

The inputs and outputs of each node are defined with the appropriate types and names, following the naming convention you specified. For example, input ports start with "in" followed by a descriptive name, and output ports start with "output" followed by a descriptive name.

The edges represent the connections between the output and input ports of the nodes. In this case, the output of "Add" is connected to the first input of "Subtract," the output of "Power" is connected to the second input of "Subtract," the output of "Subtract" is connected to the input of "Square Root," and the output of "Square Root" is connected to the input of "Distance."

This node network calculates the distance between two 3D points by performing the necessary mathematical operations. Feel free to modify the example or add more nodes and connections as needed.

**Turn this into a nodegraph with exposed inputs and outputs**

Sure! Here's an updated version of the example where the nodes are placed within a node graph, and inputs are added to the node graph to accept the two points as inputs:

```json
{
  "name": "DistanceCalculator",
  "nodes": [
    {
      "id": "nodeGraph1",
      "category": "NodeGraph",
      "group": "Math",
      "implementation": "distance_calculator.nodegraph",
      "inputs": [
        {
          "id": "input1",
          "name": "pointA",
          "type": "vector3"
        },
        {
          "id": "input2",
          "name": "pointB",
          "type": "vector3"
        }
      ],
      "outputs": [
        {
          "id": "output1",
          "name": "distance",
          "type": "vector3"
        }
      ],
      "nodes": [
        {
          "id": "node1",
          "category": "Add",
          "group": "Math",
          "implementation": "addition.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inVectorA",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inVectorB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputVector",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node2",
          "category": "Subtract",
          "group": "Math",
          "implementation": "subtraction.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inVectorA",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inVectorB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputVector",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node3",
          "category": "Power",
          "group": "Math",
          "implementation": "power.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inValue",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inPower",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputValue",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node4",
          "category": "Square Root",
          "group": "Math",
          "implementation": "square_root.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inValue",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputValue",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node5",
          "category": "Distance",
          "group": "Math",
          "implementation": "distance.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inPointA",
              "type": "vector3"
            },


            {
              "id": "in2",
              "name": "inPointB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputDistance",
              "type": "vector3"
            }
          ]
        }
      ],
      "edges": [
        {
          "from": {
            "nodeId": "node1",
            "portId": "output1"
          },
          "to": {
            "nodeId": "node2",
            "portId": "in1"
          }
        },
        {
          "from": {
            "nodeId": "node3",
            "portId": "output1"
          },
          "to": {
            "nodeId": "node2",
            "portId": "in2"
          }
        },
        {
          "from": {
            "nodeId": "node2",
            "portId": "output1"
          },
          "to": {
            "nodeId": "node4",
            "portId": "in1"
          }
        },
        {
          "from": {
            "nodeId": "node4",
            "portId": "output1"
          },
          "to": {
            "nodeId": "node5",
            "portId": "in1"
          }
        }
      ]
    }
  ]
}
```

In this updated example, a node graph with the id "nodeGraph1" has been created. The node graph has two input ports: "pointA" and "pointB" of type "vector3" which represent the two points to compute the distance between. The node graph also has one output port named "distance" of type "vector3" which represents the calculated distance.

Inside the node graph, the previously defined nodes "Add," "Subtract," "Power," "Square Root," and "Distance" are included. The connections between the nodes remain the same as in the previous example.

By encapsulating the nodes within a node graph, you can now treat the entire network as a single entity with its own inputs and outputs.

**Write as XML Now**

Certainly! Here's an XML representation of the node graph and its connections:

```xml
<NodeGraph name="DistanceCalculator">
  <Nodes>
    <Node id="nodeGraph1" category="NodeGraph" group="Math" implementation="distance_calculator.nodegraph">
      <Inputs>
        <Port id="input1" name="pointA" type="vector3" />
        <Port id="input2" name="pointB" type="vector3" />
      </Inputs>
      <Outputs>
        <Port id="output1" name="distance" type="vector3" />
      </Outputs>
      <Nodes>
        <Node id="node1" category="Add" group="Math" implementation="addition.node">
          <Inputs>
            <Port id="in1" name="inVectorA" type="vector3" />
            <Port id="in2" name="inVectorB" type="vector3" />
          </Inputs>
          <Outputs>
            <Port id="output1" name="outputVector" type="vector3" />
          </Outputs>
        </Node>
        <Node id="node2" category="Subtract" group="Math" implementation="subtraction.node">
          <Inputs>
            <Port id="in1" name="inVectorA" type="vector3" />
            <Port id="in2" name="inVectorB" type="vector3" />
          </Inputs>
          <Outputs>
            <Port id="output1" name="outputVector" type="vector3" />
          </Outputs>
        </Node>
        <Node id="node3" category="Power" group="Math" implementation="power.node">
          <Inputs>
            <Port id="in1" name="inValue" type="vector3" />
            <Port id="in2" name="inPower" type="vector3" />
          </Inputs>
          <Outputs>
            <Port id="output1" name="outputValue" type="vector3" />
          </Outputs>
        </Node>
        <Node id="node4" category="Square Root" group="Math" implementation="square_root.node">
          <Inputs>
            <Port id="in1" name="inValue" type="vector3" />
          </Inputs>
          <Outputs>
            <Port id="output1" name="outputValue" type="vector3" />
          </Outputs>
        </Node>
        <Node id="node5" category="Distance" group="Math" implementation="distance.node">
          <Inputs>
            <Port id="in1" name="inPointA" type="vector3" />
            <Port id="in2" name="inPointB" type="vector3" />
          </Inputs>
          <Outputs>
            <Port id="output1" name="outputDistance" type="vector3" />
          </Outputs>
        </Node>
      </Nodes>
      <Edges>
        <Edge>
          <From nodeId="node1" portId="output1" />
          <To nodeId="node2" portId="in1" />
        </Edge>
        <Edge>
          <From nodeId="node3" portId="output1" />
          <To nodeId="node2" portId="in2" />
        </Edge>
        <Edge>
          <From nodeId="node2" portId="output1" />
          <To nodeId="node4" portId="in1" />
        </Edge>
        <Edge>
          <From nodeId="node4" portId="output1" />
          <To nodeId="node5" portId="in1

" />
        </Edge>
      </Edges>
    </Node>
  </Nodes>
</NodeGraph>
```

In this XML representation, the node graph is wrapped inside a `<NodeGraph>` element with the name "DistanceCalculator". The nodes are contained within the `<Nodes>` element, and each node is represented by a `<Node>` element with its respective attributes such as id, category, group, and implementation.

The inputs and outputs of each node are defined within their respective `<Inputs>` and `<Outputs>` elements. The connections between nodes are represented by the `<Edges>` element, with each connection defined by an `<Edge>` element. The `From` and `To` attributes of the `<Edge>` element specify the source and target ports of the connection, respectively.

Note that the XML structure may vary depending on the specific XML schema or requirements of your implementation, but this representation should give you a starting point for encoding the node graph as XML.

Certainly! Here's the updated JSON representation of the example node network using the modified schema with "/" as path separators for the edge paths:

```json
{
  "name": "DistanceCalculator",
  "nodes": [
    {
      "id": "nodeGraph1",
      "category": "NodeGraph",
      "group": "Math",
      "implementation": "distance_calculator.nodegraph",
      "inputs": [
        {
          "id": "input1",
          "name": "pointA",
          "type": "vector3"
        },
        {
          "id": "input2",
          "name": "pointB",
          "type": "vector3"
        }
      ],
      "outputs": [
        {
          "id": "output1",
          "name": "distance",
          "type": "vector3"
        }
      ],
      "nodes": [
        {
          "id": "node1",
          "category": "Add",
          "group": "Math",
          "implementation": "addition.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inVectorA",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inVectorB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputVector",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node2",
          "category": "Subtract",
          "group": "Math",
          "implementation": "subtraction.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inVectorA",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inVectorB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputVector",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node3",
          "category": "Power",
          "group": "Math",
          "implementation": "power.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inValue",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inPower",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputValue",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node4",
          "category": "Square Root",
          "group": "Math",
          "implementation": "square_root.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inValue",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputValue",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node5",
          "category": "Distance",
          "group": "Math",
          "implementation": "distance.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inPointA",
              "type": "vector3"
            },
            {
              "id": "in2

",
              "name": "inPointB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputDistance",
              "type": "vector3"
            }
          ]
        }
      ],
      "edges": [
        {
          "from": "node1/output1",
          "to": "node2/in1"
        },
        {
          "from": "node3/output1",
          "to": "node2/in2"
        },
        {
          "from": "node2/output1",
          "to": "node4/in1"
        },
        {
          "from": "node4/output1",
          "to": "node5/in1"
        }
      ]
    }
  ]
}
```

In this JSON representation, the edges are defined using the "from" and "to" properties, where the value is the string path to the source and target ports, respectively. The paths are constructed by concatenating the node ID, "/", and the port ID.

Note that the schema has been modified accordingly to accommodate this change in the edge representation.

*Add more rules to specify edges at the correct level*

Apologies for the confusion. Based on your clarification, here's the updated JSON representation where the edges connecting nodes inside the node graph remain inside the graph, while the edges connecting nodes inside the graph to those outside the graph are specified at the same level as the nodes outside the graph:

```json
{
  "name": "DistanceCalculator",
  "nodes": [
    {
      "id": "nodeGraph1",
      "category": "NodeGraph",
      "group": "Math",
      "implementation": "distance_calculator.nodegraph",
      "inputs": [
        {
          "id": "input1",
          "name": "pointA",
          "type": "vector3"
        },
        {
          "id": "input2",
          "name": "pointB",
          "type": "vector3"
        }
      ],
      "outputs": [
        {
          "id": "output1",
          "name": "distance",
          "type": "vector3"
        }
      ],
      "nodes": [
        {
          "id": "node1",
          "category": "Add",
          "group": "Math",
          "implementation": "addition.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inVectorA",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inVectorB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputVector",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node2",
          "category": "Subtract",
          "group": "Math",
          "implementation": "subtraction.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inVectorA",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inVectorB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputVector",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node3",
          "category": "Power",
          "group": "Math",
          "implementation": "power.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inValue",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inPower",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputValue",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node4",
          "category": "Square Root",
          "group": "Math",
          "implementation": "square_root.node",
          "inputs": [
            {
              "id": "in1",
              "name": "inValue",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputValue",
              "type": "vector3"
            }
          ]
        },
        {
          "id": "node5",
          "category": "Distance",
          "group": "Math",
          "implementation": "distance.node",
          "inputs": [
            {
              "id": "in

1",
              "name": "inPointA",
              "type": "vector3"
            },
            {
              "id": "in2",
              "name": "inPointB",
              "type": "vector3"
            }
          ],
          "outputs": [
            {
              "id": "output1",
              "name": "outputDistance",
              "type": "vector3"
            }
          ]
        }
      ],
      "edges": [
        {
          "from": "node1/output1",
          "to": "node2/in1"
        },
        {
          "from": "node3/output1",
          "to": "node2/in2"
        },
        {
          "from": "node2/output1",
          "to": "node4/in1"
        },
        {
          "from": "node4/output1",
          "to": "node5/in1"
        }
      ]
    }
  ],
  "edges": [
    {
      "from": "constant1/output",
      "to": "nodeGraph1/node1/in1"
    },
    {
      "from": "constant2/output",
      "to": "nodeGraph1/node1/in2"
    }
  ],
  "nodes": [
    {
      "id": "constant1",
      "category": "Constant",
      "group": "Math",
      "implementation": "constant.node",
      "outputs": [
        {
          "id": "output",
          "name": "value",
          "type": "vector3",
          "value": "1,0,0"
        }
      ]
    },
    {
      "id": "constant2",
      "category": "Constant",
      "group": "Math",
      "implementation": "constant.node",
      "outputs": [
        {
          "id": "output",
          "name": "value",
          "type": "vector3",
          "value": "0,1,0"
        }
      ]
    }
  ]
}
```

In this updated JSON representation, the edges connecting the nodes inside the node graph (`node1`, `node2`, `node3`, `node4`, `node5`) remain inside the `nodeGraph1` object, while the edges connecting the nodes inside the graph to those outside the graph are specified at the same level as the nodes outside the graph (`constant1` and `constant2`).

**Note**

The is "runtime" XML which is more compact: nodes/nodegraphs, ports and edges only.

# Using the JSON schema to graph graphs from code ?!?!?!
